<a href="https://colab.research.google.com/github/antoniocfetngnu/InteligArtificial1/blob/main/Clasificaciones/onevsAll/CalderonEAntonio_reg_log_onevsall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Descripcion Dataset (Predicción de magnitud de terremotos)

Este dataset nos entrega información pertinente a terremotos, siendo uno de esos datos la **magnitud** de un terremoto que se utilizará como **Label Y** **redondeandolos a valores enteros** para tener categorías comunes entre terremotos, las columnas que se tienen son:

**Time**: Timestamp of the earthquake event.

**Latitude**: Geographic coordinate specifying the north-south position.

**Longitude**: Geographic coordinate specifying the east-west position.

**Depth**: Depth of the earthquake in kilometers.

**Mag**: Magnitude of the earthquake.

**MagType**: Type of magnitude measurement.

**Nst**: Number of seismic stations that reported the earthquake.

**Gap**: The gap between different seismic stations' coverage.

**Dmin**: Minimum distance to the earthquake epicenter for the nearest station.

**Rms**: Root Mean Square of the earthquake's amplitude spectrum.

**Net**: Network reporting the earthquake.

**Id**: Unique identifier for the earthquake event.

**Updated**: Timestamp indicating when the earthquake information was last updated.

**Place**: Location description of the earthquake.

**Type**: Type of seismic event (e.g., earthquake).

**HorizontalError**: Horizontal error in location determination.

**DepthError**: Error in depth determination.

**MagError**: Error in magnitude determination.

**MagNst**: Number of seismic stations used to calculate the magnitude.

**Status**: Status of the earthquake event (e.g., reviewed).

**LocationSource**: Source reporting the earthquake location.

**MagSource**: Source reporting the earthquake magnitude.

A continuación se realiza el ejercicio analizando el dataset, limpiandolo y dejarlo listo para hacer las predicciones onevsAll

# Clasificación multiclase

##Importacion de librerías

Primero importamos todas las librerías que necesitamos

In [104]:
import pandas as pd
import numpy as np
from scipy import optimize
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import random
from sklearn.metrics import accuracy_score

Ahora importamos el dataset como dataframe de pandas

In [105]:
df = pd.read_csv('earthquakes_2023_global.csv')

#Limpieza y acomodo de Dataset
Despues de analizar el dataset, debemos eliminar algunas columnas que no son necesarias (columnas de ID, columnas que tienen el mismo valor en todo,etc)

##status
En este caso **status** tiene el mismo valor en toda la columna, por tanto no es relevante y se elimina.

##type
Tambien **type**, ya que el objetivo es **predecir la magnitud de los terremotos**, en el dataset hay una minuscula (menores a 100 e incluso 10) datos que no corresponde a terremotos, por tanto como se decidió filtrar los datos a solo un mismo tipo (terremotos) entonces esta columna ya no tiene un valor variable por tanto también se eliminó.

##Columnas con texto
Las columnas a las que se les aplico **pd.factorize** son columnas las cuales tienen como valores texto y categorías, por tanto, lo que se hizo fue convertirlos a un equivalente numérico con un valor distinto para cada ocurrencia de distinto nombre, de esta forma seguirá categorizado pero numéricamente.

Por tanto, se añadió nuevas columnas con el equivalente numérico, entonces las columnas fuente, también son eliminadas ya que tienen su representación numerica en nuevas columnas.

##Columnas con formato ISO de fecha

El dataset cuenta con 2 columnas, una que indica cuando se registró el terremoto y otra columna que indica cuando se actualizó el evento por ultima vez.

En este caso, nos interesa extraer el mes donde ocurrio el evento, con la intención de predecir temporadas donde ocurren los terremotos, por tanto se extrae el mes de ambas columnas y se crean 2 nuevas columnas que los contienen, y se eliminan las columnas originales.

##Salida (label) 'mag'

Para la salida lo que se hizo fue "redondear" las magnitudes a su entero mas cercano (priorizando cuando sea el caso de .5 al entero superior) para poder tener categorías útiles para el ejercicio sin afectar mucho a la validez de los datos.

In [106]:
pd.set_option('display.max_columns', None)
df.drop('type', axis=1, inplace=True)
df.drop('status', axis=1, inplace=True)
df['magType_code'] = pd.factorize(df['magType'])[0] + 1
df['location_code'] = pd.factorize(df['locationSource'])[0] + 1
df['magSource_code'] = pd.factorize(df['magSource'])[0] + 1
df['net_code'] = pd.factorize(df['net'])[0]+1
df.drop('magType', axis=1, inplace=True)
df.drop('locationSource', axis=1, inplace=True)
df.drop('magSource', axis=1, inplace=True)
df.drop('id', axis=1, inplace=True)
df.drop('net', axis=1, inplace=True)

#convertir el formato ISO a fecha con datetime de pandas
df['time'] = pd.to_datetime(df['time'].str.replace('Z', '+00:00'), format='%Y-%m-%dT%H:%M:%S.%f%z')
df['updated'] = pd.to_datetime(df['updated'].str.replace('Z', '+00:00'), format='%Y-%m-%dT%H:%M:%S.%f%z')

# Extraer el mes de la fecha y crear una nueva columna "Month"
df['Month'] = df['time'].dt.month

df['MonthUpdated'] = df['updated'].dt.month

df.drop('time', axis=1, inplace=True)
df.drop('updated', axis=1, inplace=True)
df.drop('place', axis=1, inplace=True)
#comprobada la increible correlacion entre location_code, magSource_code y net_code entonces se decidio mantener solo una ya que expresan lo mismo
df.drop('location_code', axis=1, inplace=True)
df.drop('net_code', axis=1, inplace=True)
#redondeamos y en los casos en que sea .5 se redonde al mayor no al menor
df['mag'] = df['mag'].apply(lambda x: np.ceil(x) if x - int(x) == 0.5 else np.round(x))

Una vez, acomodado los tipos de datos de todas las columnas satisfactoriamente, aplicamos **df.describe()** para que nos muestre un resumen de todas nuestras columnas, mediante esto podremos identificar si algunas columnas tienen NANs (datos faltantes) o si ya estuviera preparado para la regresión logistica onevsAll

In [107]:
print(df.describe())

           latitude     longitude         depth           mag           nst  \
count  26642.000000  26642.000000  26642.000000  26642.000000  25227.000000   
mean      16.852798    -11.487497     67.491224      4.084341     42.571332   
std       30.389200    130.053399    116.762456      0.839589     37.662352   
min      -65.849700   -179.998700     -3.370000      3.000000      0.000000   
25%       -6.415275   -149.608650     10.000000      3.000000     19.000000   
50%       18.884167    -64.811833     21.998000      4.000000     30.000000   
75%       41.827950    126.965100     66.833000      5.000000     52.000000   
max       86.593900    179.999400    681.238000      8.000000    423.000000   

                gap          dmin           rms  horizontalError  \
count  25225.000000  24776.000000  26642.000000     25093.000000   
mean     124.930971      2.692908      0.581575         7.017267   
std       67.430145      4.043568      0.256276         4.072365   
min        8.000

Gracias a la respuesta anterior, se puede observar que el atributo **count** de algunas columnas no coincide, eso significa que su cantidad de valores es menor posiblemente debido a la presencia de casillas vacías **NANs**, por tanto debemos ver la forma de tratar estos campos.

In [108]:
nans_en_columna = df['latitude'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'latitude': {nans_en_columna}")
nans_en_columna1 = df['longitude'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'longitude': {nans_en_columna1}")
nans_en_columna2 = df['depth'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'depth': {nans_en_columna2}")
nans_en_columna3 = df['mag'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'mag': {nans_en_columna3}")
nans_en_columna4 = df['nst'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'nst': {nans_en_columna4}")
nans_en_columna5 = df['gap'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'gap': {nans_en_columna5}")
nans_en_columna6 = df['dmin'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'dmin': {nans_en_columna6}")
nans_en_columna7 = df['rms'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'rms': {nans_en_columna7}")
nans_en_columna8 = df['horizontalError'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'horizontalError': {nans_en_columna8}")
nans_en_columna9 = df['depthError'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'depthError': {nans_en_columna9}")
nans_en_columna10 = df['magError'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'magError': {nans_en_columna10}")
nans_en_columna11 = df['magNst'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'magNst': {nans_en_columna11}")
nans_en_columna12 = df['magType_code'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'magType_code': {nans_en_columna12}")
nans_en_columna13 = df['magSource_code'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'magSource_code': {nans_en_columna13}")
nans_en_columna14 = df['Month'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'Month': {nans_en_columna14}")
nans_en_columna15 = df['MonthUpdated'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'MonthUpdated': {nans_en_columna15}")

Cantidad de NaNs en la columna 'latitude': 0
Cantidad de NaNs en la columna 'longitude': 0
Cantidad de NaNs en la columna 'depth': 0
Cantidad de NaNs en la columna 'mag': 0
Cantidad de NaNs en la columna 'nst': 1415
Cantidad de NaNs en la columna 'gap': 1417
Cantidad de NaNs en la columna 'dmin': 1866
Cantidad de NaNs en la columna 'rms': 0
Cantidad de NaNs en la columna 'horizontalError': 1549
Cantidad de NaNs en la columna 'depthError': 0
Cantidad de NaNs en la columna 'magError': 1672
Cantidad de NaNs en la columna 'magNst': 1577
Cantidad de NaNs en la columna 'magType_code': 0
Cantidad de NaNs en la columna 'magSource_code': 0
Cantidad de NaNs en la columna 'Month': 0
Cantidad de NaNs en la columna 'MonthUpdated': 0


Gracias a la ejecución del código anterior, podemos verificar que precisamente el problema es la ausencia de datos en ciertas columnas de los ejemplos del dataset.

##Llenado de valores NAN con  .mean()
Uno de los métodos para tratar estos valores NAN, es aplicar el .mean() (promedio) de los valores de la columna y rellenar los NAN con dicho valor, para este caso, se utilizará esta técnica, a las columnas identificadas con NANs anteriormente.

In [109]:
mean_nst = df["nst"].mean()
df["nst"] = df["nst"].fillna(mean_nst)

mean_gap = df["gap"].mean()
df["gap"] = df["gap"].fillna(mean_gap)

mean_dmin = df["dmin"].mean()
df["dmin"] = df["dmin"].fillna(mean_dmin)

mean_hError = df["horizontalError"].mean()
df["horizontalError"] = df["horizontalError"].fillna(mean_hError)

mean_magError= df["magError"].mean()
df["magError"] = df["magError"].fillna(mean_magError)

mean_magNst= df["magNst"].mean()
df["magNst"] = df["magNst"].fillna(mean_magNst)

##Comprobación
Una vez aplicada la media, comprobamos nuevamente si aún contamos con NaNs en las columnas:

In [110]:
nans_en_columna = df['latitude'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'latitude': {nans_en_columna}")
nans_en_columna1 = df['longitude'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'longitude': {nans_en_columna1}")
nans_en_columna2 = df['depth'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'depth': {nans_en_columna2}")
nans_en_columna3 = df['mag'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'mag': {nans_en_columna3}")
nans_en_columna4 = df['nst'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'nst': {nans_en_columna4}")
nans_en_columna5 = df['gap'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'gap': {nans_en_columna5}")
nans_en_columna6 = df['dmin'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'dmin': {nans_en_columna6}")
nans_en_columna7 = df['rms'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'rms': {nans_en_columna7}")
nans_en_columna8 = df['horizontalError'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'horizontalError': {nans_en_columna8}")
nans_en_columna9 = df['depthError'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'depthError': {nans_en_columna9}")
nans_en_columna10 = df['magError'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'magError': {nans_en_columna10}")
nans_en_columna11 = df['magNst'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'magNst': {nans_en_columna11}")
nans_en_columna12 = df['magType_code'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'magType_code': {nans_en_columna12}")
nans_en_columna13 = df['magSource_code'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'magSource_code': {nans_en_columna13}")
nans_en_columna14 = df['Month'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'Month': {nans_en_columna14}")
nans_en_columna15 = df['MonthUpdated'].isnull().sum()
print(f"Cantidad de NaNs en la columna 'MonthUpdated': {nans_en_columna15}")
print(df.describe())

Cantidad de NaNs en la columna 'latitude': 0
Cantidad de NaNs en la columna 'longitude': 0
Cantidad de NaNs en la columna 'depth': 0
Cantidad de NaNs en la columna 'mag': 0
Cantidad de NaNs en la columna 'nst': 0
Cantidad de NaNs en la columna 'gap': 0
Cantidad de NaNs en la columna 'dmin': 0
Cantidad de NaNs en la columna 'rms': 0
Cantidad de NaNs en la columna 'horizontalError': 0
Cantidad de NaNs en la columna 'depthError': 0
Cantidad de NaNs en la columna 'magError': 0
Cantidad de NaNs en la columna 'magNst': 0
Cantidad de NaNs en la columna 'magType_code': 0
Cantidad de NaNs en la columna 'magSource_code': 0
Cantidad de NaNs en la columna 'Month': 0
Cantidad de NaNs en la columna 'MonthUpdated': 0
           latitude     longitude         depth           mag           nst  \
count  26642.000000  26642.000000  26642.000000  26642.000000  26642.000000   
mean      16.852798    -11.487497     67.491224      4.084341     42.571332   
std       30.389200    130.053399    116.762456    

Como se puede observar se aplicó correctamente y ya no tenemos NANs en las columnas

#Separación en entrenamiento y prueba
Ahora, con el dataset listo, debemos separar los datos de manera coherente
##Tomar el 20% de los datos de cada ocurrencia en Y(label)
Entonces, utilizando train_test_split con stratify y test_size=0.2 aseguramos que se tome el 20% de cada ocurrencia en Y, además la semilla aleatoria es 42, por si quisieramos replicar la pseudo aleatorieadad al correr nuevamente el código.

También se realizó la verificación de proporciones de los datos en y_train y y_test y vemos que la proporcion es similar, en algunos casos no es exacto ya que por ejemplo para terremotos de magnitud 8, no se tiene muchos datos (magnitud 8 datos <100)


In [111]:
y1 = df['mag']
X1 = df.drop(columns=['mag'])

print(X1.head())
print('-'*20)
print(y1.head())
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, stratify=y1, random_state=42)

# Verificar la proporción de clases en los conjuntos de entrenamiento y prueba
print("Proporción de clases en y_train:")
print(y_train.value_counts(normalize=True))

print("\nProporción de clases en y_test:")
print(y_test.value_counts(normalize=True))

   latitude  longitude   depth   nst    gap    dmin   rms  horizontalError  \
0   52.0999   178.5218  82.770  14.0  139.0  0.8700  0.18             8.46   
1    7.1397   126.7380  79.194  32.0  104.0  1.1520  0.47             5.51   
2   19.1631   -66.5251  24.000  23.0  246.0  0.8479  0.22             0.91   
3   -4.7803   102.7675  63.787  17.0  187.0  0.4570  0.51            10.25   
4   53.3965  -166.9417  10.000  19.0  190.0  0.4000  0.31             1.41   

   depthError  magError  magNst  magType_code  magSource_code  Month  \
0      21.213     0.097    14.0             1               1      1   
1       7.445     0.083    43.0             2               1      1   
2      15.950     0.090    16.0             3               2      1   
3       6.579     0.238     5.0             2               1      1   
4       1.999     0.085    18.0             1               1      1   

   MonthUpdated  
0             3  
1             3  
2             3  
3             3  
4       

Una vez separados los datos en entrenamiento y prueba, debemos **normalizar las X**, entonces instanciamos un **objeto scaler**, mediante el cual podemos **estandarizar**(usar la media y la desviacion estandar para **normalizar**) como usualmente hacíamos con la función definida por nosotros(ojo: se realizó una **comparación** de las salidas obtenidas **con este método** y con la **función original del ejemplo** y se obtiene exactamente el **mismo resultado**, pero sin embargo de esta forma se tiene la **ventaja** de estandarizar las **X de prueba** con la desviación y media de **X de entrenamiento** para que ambos posean la **misma estandarización aplicada**.

In [112]:
# Normalizar características solo en los conjuntos de entrenamiento y prueba
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(X_train_scaled)
print('-'*30)
print(X_test_scaled)

# Configurar la matriz adecuadamente, y agregar una columna de unos que corresponde al termino de intercepción.
m, n = X_train_scaled.shape

[[ 0.0568953   1.21179003 -0.49256614 ... -0.39275517  1.3093833
   0.88942869]
 [-2.39244322 -0.12026503 -0.26384398 ... -0.39275517  0.45001357
   0.56832437]
 [-0.30944051  1.0597022  -0.49166697 ... -0.39275517 -0.98226932
  -1.03719725]
 ...
 [-2.44207669 -0.43145249 -0.49256614 ... -0.39275517 -0.40935617
  -0.07388428]
 [ 0.10033819  1.20450601 -0.49256614 ... -0.39275517  1.3093833
   1.21053302]
 [ 0.034744   -0.42539004 -0.46070997 ...  0.1310085  -0.69581274
  -1.35830157]]
------------------------------
[[-0.85158282  1.29090079 -0.49256614 ... -0.39275517 -1.55518248
  -1.67940589]
 [-0.78476283  1.08786482  0.24353403 ... -0.39275517  0.73647014
   0.88942869]
 [ 1.2039696  -1.16862935 -0.27847897 ... -0.39275517 -1.55518248
  -1.67940589]
 ...
 [ 1.1739105  -0.15477281 -0.49256614 ... -0.39275517  0.45001357
   0.56832437]
 [ 1.16919748 -1.21403817 -0.27847897 ... -0.39275517 -1.2687259
  -1.35830157]
 [ 1.13291152 -1.27656631 -0.27847897 ... -0.39275517 -1.2687259
  -1.

Ya que el método onevsAll es la aplicación de reg. Logística pero para varias categorías, debemos definir la sigmoide como hicimos en el caso anterior:

In [113]:
def sigmoid(z):

    return 1.0 / (1.0 + np.exp(-z))

Se define la función de costo pero en este caso aumentando lambda para evitar sobreajustes y por tanto que el modelo no tome en cuenta o minimice el ruido de datos muy dispersos.

In [114]:
def lrCostFunction(theta, X, y, lambda_):

    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))
    # print('valor de h ================')
    # print(h)

    temp = theta
    temp[0] = 0

    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp
    # print('valores intermedios J----------------')
    # print(J)
    # print(grad)
    return J, grad

Definimos la función de oneVsAll y en este caso no usaremos nuestra función de descenso por la gradiente, sino usaremos **optimize.minimize** con su propio **metodo 'CG'**que **reemplaza** al descenso por la gradiente que usabamos en anteriores ejercicios.

Además, debido a que usaremos **np.arange**, los valores de c irán variando de 0 hasta el rango máximo -1 (num_labels), la **ventaja de arange** a diferencia de **range de python** sin numpy, puede manejar **vectores** y en este caso como calcularemos las thithas de varias categorías nos es muy util.

Sin embargo, debido a que se compara **y==c** en los parámetros de **optimize.minimize**, entonces lo que hará es buscar **coincidencias** de **y** a la **c** actual y **tomará solo las coincidencias** para entrenar a ese conjunto, de ahi el nombre de onevsAll, y así sucesivamente hasta salir del ciclo for. Es por eso que **las magnitudes de salida de nuestra Y debemos acomodarlas** al mismo rango de **c** para que pueda coincidir **y==c**

In [115]:
def oneVsAll(X, y, num_labels, lambda_):

    # algunas variables utiles
    m, n = X.shape
    print('------------------')
    print('valor n:',n)
    print('valor m:', n)
    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        print("valor de c:",c)
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

Entonces, primero debemos verificar cuantas categorías tenemos en nuestro **label Y**:

In [116]:
print(y_train.unique())

[5. 4. 3. 6. 7. 8.]


Gracias a lo anterior podemos ver que tenemos **6 categorías** (magnitudes de terremotos) pero no van del 0 al 5, por tanto **mapearemos** de 0 a 5 para **coincidir con los valores de c**

In [117]:
mapeo_etiquetas = {3: 0, 4: 1, 5: 2, 6: 3, 7: 4, 8: 5}
y_train_mapped = y_train.map(mapeo_etiquetas)       #mapeamos a valores del 0 al 5 para el onevsAll
y_train_mapped_array = y_train_mapped.values        #lo convertimos en un numpy array para poder usarlo en nuestras funciones

num_labels = len(np.unique(y_train_mapped_array))
print(num_labels)

6


Ahora, ya estamos preparados para calculas las thitas correspondientes mediante **oneVsAll** para nuestro dataset y tenemos:

In [118]:
lambda_ = 0.1
all_theta = oneVsAll(X_train_scaled, y_train_mapped_array, num_labels, lambda_)
print('------------------------------------Salidaaaaaaaaa')
print(all_theta.shape)

print(all_theta)


------------------
valor n: 15
valor m: 15
valor de c: 0
valor de c: 1
valor de c: 2
valor de c: 3
valor de c: 4
valor de c: 5
------------------------------------Salidaaaaaaaaa
(6, 16)
[[-4.70023641e+00  2.20498833e+00 -9.44600804e-01 -8.31399694e-01
  -1.85976970e+00  3.12250308e-01 -3.05387381e+00 -1.02640606e+00
  -2.91947748e-01 -1.89349308e-01 -1.96925586e-02  5.52314355e-01
  -8.56346369e-01  8.17696319e-01  8.42866992e-01 -8.59869903e-01]
 [-8.09385132e-01  6.13282326e-02  4.07183762e-01  4.93254405e-01
  -2.59059718e-01 -1.97679447e-01 -1.22966217e-01  2.21590705e-01
   8.39279474e-03  2.57076239e-01  3.48477087e-01 -9.00501722e-01
  -2.34093090e-01 -2.61844322e-01 -7.00353011e-01  6.55264809e-01]
 [-1.27285537e+00 -9.15591631e-01  2.82074652e-01 -6.22722309e-01
   3.84178480e-01 -5.83397331e-01  8.91943722e-02  3.25225058e-01
   4.89554866e-01 -2.03456768e-01 -1.02828911e+00  4.38209886e-01
  -2.65232839e-01 -4.25518346e-01  2.82455108e-01 -2.23430247e-01]
 [-1.01007245e+01 -

In [119]:
def predictOneVsAll(all_theta, X):

    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

Una vez obtenidas las thitas, es momento de realizar las predicciones, primero definimos la funcion que filtra la concordancia de clases con su **máximo** (para saber qué conjunto de thitas corresponde a qué **clase de Y**(categoria)).

Para realizar esto debemos usar la X de prueba normalizada, entonces:

In [120]:
print(X_test_scaled.shape)
predicciones = predictOneVsAll(all_theta, X_test_scaled)
print(predicciones)

(5329, 15)
[2 1 0 ... 2 0 0]


Las predicciones (Y predicha) se guarda en **predicciones**, pero este se encuentra en formato **array de numpy**, por tanto para sacar un **accuracy score** correcto, ambos y de prueba y predicciones deben ser del mismo tipo de datos, en este caso es mas facil **volver a predicciones en un dataframe de solo una columna** ya que **y_test es un dataframe** que aún no se había transformado a un array de numpy.

###OJO IMPORTANTE
Ojo, no hay que olvidar que **mapeamos los datos de nuestra y de entrenamiento** para que **coincidan** con los valores de **c**, entonces, para hacer una **correcta comparación** de las y predichas mapeadas con las y de prueba debemos **mapear inversamente** las **y predichas** para que tengan sus valores **originales**. Entonces:

In [121]:
mapeo_etiquetas_reverse = {v: k for k, v in mapeo_etiquetas.items()}

predicciones_df = pd.DataFrame({'Predicciones': predicciones})

# Aplicar el mapeo inverso a las predicciones del DataFrame
predicciones_df['Predicciones'] = predicciones_df['Predicciones'].map(mapeo_etiquetas_reverse)

print(predicciones_df)
y_test_renumerado = y_test.reset_index(drop=True)
print(y_test_renumerado)

precision = accuracy_score(y_test, predicciones_df['Predicciones'])
print("Precisión del modelo:", precision)

      Predicciones
0                5
1                4
2                3
3                5
4                3
...            ...
5324             3
5325             5
5326             5
5327             3
5328             3

[5329 rows x 1 columns]
0       5.0
1       4.0
2       3.0
3       5.0
4       3.0
       ... 
5324    3.0
5325    4.0
5326    5.0
5327    3.0
5328    3.0
Name: mag, Length: 5329, dtype: float64
Precisión del modelo: 0.7729405141677613


Como se puede ver, el modelo tiene 0.77 de precisión, esto significa una precisión del 77% por tanto, es un modelo válido y el ejercicio se realizó correctamente.